In [22]:
# Warning 생략
import warnings
warnings.filterwarnings('ignore')
# Pandas
import pandas as pd
# Numpy
import numpy as np
# ML 저장
import joblib
# 그래프
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
# Seaborn
import seaborn as sns

# ML Data 나누기
from sklearn.model_selection import train_test_split
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
# Random Forest
from sklearn.ensemble import RandomForestClassifier
# SVM
from sklearn import svm
from sklearn.svm import SVC
# Neural Net
from sklearn.neural_network import MLPClassifier
# QDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
# AdaBoost
from sklearn.ensemble import AdaBoostClassifier
# Gaussian Process
from sklearn.gaussian_process import GaussianProcessClassifier
# KNN
from sklearn.neighbors import KNeighborsClassifier
# 회귀모델
from sklearn.neighbors import KNeighborsRegressor
# 선형회귀
from sklearn.linear_model import LinearRegression
# 다중회귀
from sklearn.preprocessing import PolynomialFeatures
# 로지스틱 회귀
from sklearn.linear_model import LogisticRegression
# Grid Search
from sklearn.model_selection import GridSearchCV
# Extra Tree
from sklearn.ensemble import ExtraTreesClassifier
# Gradient Boostring
from sklearn.ensemble import GradientBoostingClassifier
# Histogram gradient boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
# XGBoost
from xgboost import XGBClassifier
# Permutation Importance (치환중요도)
from sklearn.inspection import permutation_importance

# 평균 절대값 오차
from sklearn.metrics import mean_absolute_error

from sklearn import datasets

# 교차 검증
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
# 표준화
from sklearn.preprocessing import StandardScaler
# Lasso (L1)
from sklearn.linear_model import Lasso
# Ridge (L2)
from sklearn.linear_model import Ridge
# 확률적 경사하강법
from sklearn.linear_model import SGDClassifier

# Cluster
from sklearn.cluster import KMeans
# Voting
from sklearn.ensemble import VotingClassifier
# 정확도 측정
from sklearn.metrics import accuracy_score
# Sigmoid
from scipy.special import expit
# SoftMax
from scipy.special import softmax
# KFold
from sklearn.model_selection import StratifiedKFold

# 한글 폰트 문제 해결 
# matplotlib은 한글 폰트를 지원하지 않음
# os정보
import platform

# font_manager : 폰트 관리 모듈
# rc : 폰트 변경 모듈
from matplotlib import font_manager, rc
# unicode 설정
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")

### 앙상블(Ensemble)
- 여러개의 분류 모델을 조합해서 더 나은 성능을 내는 방법
- Decision Tree모델을 증가시켜 나온 RandomForest가 대표적임

#### Random Forest
- 부트스트랩 샘플을 사용합니다. 부트스트랩 샘플링은 중복을 허용하는 샘플링 방법
- 샘플링 후에 샘플을 다시 복구하고 다시 샘플링 하는 방법 입니다.
- 이와 같이 진행하는 이유는 결정트리에서 과대적합을 방지 할 수 있기 때문이다.
- 각 결정트리에서 나오는 확률의 합을 트리개수로 나누어 결정짓는 모델
- 특성의 개수를 제곱근으로 합니다.

In [2]:
wine = pd.read_csv('../Data/wine.csv')

In [3]:
# Feature, Target
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [5]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [6]:
# 주요 Feature 확인
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [8]:
# 부트스트랩 결정시 남는 샘플(oob : out of back)도 특성으로 구분할 수 있다.
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


> oob를 사용해도 90%의 예측이 나온다.</br>
> 따로 검증셋을 구성하지 않아도 oob로 검증세트의 역할을 대신할 수 있다.

> RandomForest는 기본값만으로도 높은 성능을 발휘하므로 자주 사용되는 모델(알고리즘)중 하나이다.

---
### Extra Tree
- 기본적으로 100개의 트리를 사용
- 노드 분할시 특성의 제곱근을 개수로 사용
- 특성의 선택을 랜덤하게 선택한다.
- 특성의 선택을 랜덤하게 하므로 속도는 랜덤포레스트보다 빠르다.

In [10]:
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [12]:
et.fit(train_input, train_target)
et.feature_importances_

array([0.20183568, 0.52242907, 0.27573525])

---
### Gradient Boostring(그레디언트 부스팅)
- 가장 유명한 알고리즘 중 하나이다.
- 경사하강법 처럼 손실함수를 사용.
- 손실함수를 보고 트리를 추가하여 최적의 값을 도출하는 방법
- 경사를 이동하면서 경사의 이동거리를 제어하는 learning-rate(default:0.1)를 사용한다.
- max depth를 3으로 제어하여 깊이가 낮으므로 과대적합 방지
- 단점은 손실함수를 보고 트리를 추가하면서 진행하는 모델이므로 병렬처리를 할 수 없다.

In [14]:
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


tree 추가를 500개까지하고 learning rate를 0.2로 변경

In [15]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [16]:
gb.fit(train_input, train_target)
gb.feature_importances_

array([0.15872278, 0.68010884, 0.16116839])

---
### 히스토그램 기반 그래디언트 부스팅(Histogram gradient boosting)
- 훈련데이터를 256개 구간으로 나누어서 훈련시키는 방법.
- 특성의 범위가 제한되어 있어 빠른 속도를 제공한다.
- 제한된 구간이므로 과대적합을 방지한다.
- 아직은 실험단계인 모델이다.

In [18]:
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


### XGBoost
- kaggle에서 많이 사용

In [19]:
# !pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.2 MB/s eta 0:00:00a 0:00:01


In [21]:
xgb = XGBClassifier(
    tree_method = 'hist',
    random_state = 42,
    use_label_encoder = False,
    eval_metrics = 'logloss' # 히스토그램 그래디언트 부스팅
)

scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

[18:10:03] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "eval_metrics" } are not used.

[18:10:03] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "eval_metrics" } are not used.

[18:10:03] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "eval_metrics" } are not used.

[18:10:03] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "eval_metrics" } are not used.

[18:10:03] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "eval_metrics" } are not used.

0.9555033709953124 0.8799326275264677


---
# 번외 기능
#### Permutation Importance(치환 중요도)
- 각 Feature별 Sample을 섞어서 계산을 한 후에 원래 Sample들과의 차이를 계산해서 차이가 많이 나는 Feature가 중요하다고 판단.
- 즉, 어느 Feature가 중요한지 파악하는 방법
- 어떤 모델에도 사용가능하며 특성을 파악하는 주요 기준으로 사용된다. (권장사항)

In [24]:
xgb.fit(train_input, train_target)
result = permutation_importance(xgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
result.importances_mean

[18:18:08] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "eval_metrics" } are not used.



array([0.11306523, 0.25508947, 0.10696556])

---
# 앙상블 정리
- 앙상블 학습은 정형데이터에서 가장 뛰어난 성능을 내는 머신러닝 알고리즘 중 하나 입니다.

#### 랜덤포레스트
: 부트스트랩 샘플 사용, 대표 앙상블 학습 알고리즘

#### 엑스트라 트리
: 결정 트리의 노드를 랜덤하게 분할함.

#### 그래디언트 부스팅
: 트리의 손실을 보완하는 식으로 얕은 결정트리를 연속하여 추가함.

#### 히스토그램 기반 그래디언트 부스팅
: 훈련 데이터를 256개 정수 구간으로 나누어 빠르고 높은 성능을 냄.

#### 기타
- XGBosst